## [Lab 4 (Part I)] BLENDS Node

In Lab #4 you will work in BLENDS, a simple cryptocurrency framework we have built for CS443. The BLENDS framework provides an overall architecture with well-defined APIs so that students can focus on core concepts of Bitcoin-like cryptocurrency and do not worry about architecting the system from scratch.

The framework makes the following assumptions:
* All nodes communicate not directly with each other, but through a broadcast server provided in the framework.
* Unless Bitcoin, here a single (priv, pub) key pair is associated with a node.  That is, we equate the concept of a pub key with a node.
* We use RSA, instead of ECDSA for public key crypto.
* We use SHA-512, instead of SHA-256. You will be using RSASSA-PKCS1_v1_5 of PyCryptodome [Modified on 2018/12/12]
* We do not use Merkle tree, but just a simple linked list with hash pointers.
* For ease of programming and balance-checking, we use an RDBMS called SQLite to store transactions and blocks.

The broadcast server of the framework does the following:
* It works as a communication center by accepting transactions and blocks from a node and broadcasting them to other nodes.
* It drops transactions and blocks if they are formatted correctly.
* [Under Construction] It runs a leaderboard.

A node in the framework is architected as follows:
* It is made up of two processes: main and CLI.
* The main process forks a miner thread in order to mine continuously.
* The CLI process is the command-line interface for typing in commands to be delivered to the node.  It is given in "cli.py".
* The CLI process communicates with the main process over HTTP.  "manager.py" implements the interface between the CLI and main processes.  This is also given.
* The main process is broken into 7 files: blockchain.py, db.py, node.py, crypto.py, miner.py, util.py, and manager.py. util.py and manager.py are given.  For the rest, students will have to fill in the function body given the specification.
 
Now Part I of Lab 4.  First, let's start with digital signature.


In [1]:
import json
from blends.node import crypto
from blends.node.crypto import load_secret_key, sign, verify, get_pk
import imp
imp.reload(crypto)

MESSAGE = "This is a test message"

## Digital Signature: crypto.py

You are asked to implement the following functions:
* Create and save to a file an RSA secret key.
* Load an RSA secret key from a file.
* Get a public key from an RSA secret key.
* Sign a message with an RSA secret key.

We use RSA-2048 for digital signature. The parameter `publicExponent` is fixed at 0x10001 for all keys in this project.

### Problem 1 : create_secret_key(fname)

Implement `create_secret_key` function in [crypto.py](/edit/node/crypto.py)

* It creates an RSA key and store it in a file, "fname". We keep it in a file so that when you stop in the middle and return, your node can continue with the same key.  You can look at the file format in "key0.json".


In [2]:
# from blends.node.crypto import create_secret_key
# from blends.node import crypto
# import imp
# imp.reload(crypto)
# create_secret_key("keylol.json")

### Problem 2 : load_secret_key(fname)

Implement `load_secret_key` function in [crypto.py](/edit/node/crypto.py)

* it returns secret key.
* it will be used in sign function below.

### Problem 3 : get_pk(sk)

Implement `get_pk` function in [crypto.py](/edit/node/crypto.py)

* It returns public key from secret key.

### Problem 4 : sign(sk, message)

Implement `sign` function in [crypto.py](/edit/node/crypto.py)

* Use SHA-512 to convert the message to a hash digest. Use UTF-8 encoding for the message.
* Sign message, i.e. encrypt the digest with your secret key, `sk`.
* `sk`  is return value of `load_secret_key` above.

In [3]:
# Test  for Problems 2 to 4; Problem 1 is not tested here.

from blends.node import crypto
import imp
imp.reload(crypto)

for i in range(3):
    key_fname = "key{}.json".format(i)
    sk = load_secret_key(key_fname)
    sig_computed = sign(sk, MESSAGE)
    ## print(sig_computed, end=", ")
    sig_fname = "sig{}.json".format(i)
    with open(sig_fname, "r") as sig_f:
        data = json.loads(sig_f.readline())
        sig = data["sig"]
        if sig_computed == sig:
            print("PASS")
        else:
            print("FAIL\nExpected: {}\n     Got: {}", sig, sig_computed)

PASS
PASS
PASS


### Problem 5 : verify(pk, msg, sig)

Next, Implement `verify` function in [crypto.py](/edit/node/crypto.py)

* it takes a public key `pk`, a message `msg`, and a signature `sig`.


In [4]:
sk2 = load_secret_key('key0.json')
msg2 = '{"amount": 1, "receiver": "0xafc048858bbf511c56a5a3bfd866211a4263570ad5ac47c6596be0324dfae0ccdc8d04d3e576390462d554a0873724cbbca4246632074a24eb5cac62b3150d1cacf7b11e0e0507d7b3f1ab0253539c701db8e04ff465567add08c34d930779d278fc37245e506a3528af318b1da611ddb1f6e160b776b0c10af1f6e4894bebc985a84c2dd72dbea7363448a322b8a4f480de1167c2901e288767098ed0201f5cc73ec2328de85a2f0296f58153e1adabfd99bf7c12447697983b2940c6fc2f6f9db21769afc9cb1579f651786242267311609a95313d8a3ebf76fe2743507c9017feabc4d49c601a6b00d3107fcc1f7c5358f14a83267970b5973233501fb3df", "sender": "0xb7054a9125f2f677f174130d145a4d76c64ad1e49534bd61fecabcfac309a6bea0dc40d256d086be844c1b12813bf9ed7a79faa9c2559a5af4b07b789432804286a43d64c7c4c9f3fa21353e3064bcf8dd63154862265511cd438e01470b9eda36906ae7de1d40621b78f4302d6cec9211f3da8e537a1043ac91d0bd7d417062477bb6c21761ec15091526f034ad7a36f161cb637a0282d832b236eb4be4291a0b9e55de4664a511e3418bda4300940cc4fcf95ee610a39efc256b6e9d99bd356e412477c8cd8ef704ebab0d904a865a94f42ae775eae7ea7fe36a404a2f5230bd8b5e73f0c1d66d0cd87fd65857e1bdc58620255aca68bac8eff78132532e31", "timestamp": "2018-12-12T10:53:33"}'
print(sign(sk2, msg2))

a431638b5b8ee4ae6a5662a2058b0adad263cd4c14f7fc16088033c39750daf0eabff8334039e6cf09652f3cac0f4a4ba7fb24e09fdb4e45855905b5b3784a191d88251daf277fb77f7a13f90f1feeea10af51e071c263a6151e7fd44cfb19ec471bdf12dcf25cbbd48737386eea46e274f442eb77ea174699181c130c4ad32800fff9b311945a4058cc12f4fe1ea10061103786b359a6ae274445bf266c50d52ba3fe1671fc4a14c16b90ae29219941af47e6d56b2b0a65e125ee78fa70d881906abe56b1b08b6cb295d0db08dbb9092064e89252884ed683060126c5fd6f1f36aaa00baf6d4993a86a7d67385312e1cbed356180db619c7b077e67b7f10c3e


In [5]:
# Test  for Problem 5
WRONG_SIG = "wrong signature"

for i in range(3):
    key_fname = "key{}.json".format(i)
    sk = load_secret_key(key_fname)
    pk = get_pk(sk)
    #print(sk.n)
    #print(sk.e)
    #print(sk.d)
    sig_fname = "sig{}.json".format(i)
    with open(sig_fname, "r") as sig_f:
        data = json.loads(sig_f.readline())
        sig = data["sig"]
        #print(sig)
        if verify(pk, MESSAGE, sig):
            print("PASS")
        else:
            print("FAIL")


PASS
PASS
PASS



### Problem 6 : get_hash(msg)

To send a block to a peer, you need to format the hash digest into a hexadecimal number string.

* Implement `get_hash` function in [crypto.py](/edit/node/crypto.py).
